In [5]:
using DataFrames, CSV, StringEncodings, Dictionaries, Dates, StatsBase, HTTP, JSON, ProgressBars

In [42]:
yr = 2025
datadir = joinpath("..", "Data", string(yr))

"../Data/2025"

In [43]:
seasondata = DataFrame(CSV.File(joinpath(datadir, "dataset_with_odds_and_performance.csv")))

if eltype(seasondata.game_date) != Date
    seasondata.game_date = Date.(seasondata.game_date, dateformat"m/d/y") .+ Dates.Year(2000)
end

In [ ]:
allgames = unique(seasondata.game_pk)

cnames = ["game_pk", "game_date", "home_odds", "home_id", "home_name", "home_winner", "home_win_pct_to_date", "home_last5_o_hardhit",
        "home_last5_o_launch_speed", "home_last5_o_launch_angle", "home_last5_o_xwoba",
        "home_last_p_hardhit", "home_last_p_launch_speed", "home_last_p_launch_angle", 
        "home_last_p_xwoba", "away_odds", "away_id", "away_name", "away_winner", "away_win_pct_to_date", "away_last5_o_hardhit",
        "away_last5_o_launch_speed", "away_last5_o_launch_angle", "away_last5_o_xwoba",
        "away_last_p_hardhit", "away_last_p_launch_speed", "away_last_p_launch_angle", 
        "away_last_p_xwoba"]

finalset = DataFrame([name => [] for name in cnames])

usefulcols = [:player_id, :player_name, :winner, :win_pct_to_date, :last5_o_hardhit_percent,
                :last5_o_launch_speed, :last5_o_launch_angle, :last5_o_xwoba, 
                :last_p_hardhit_percent, :last_p_launch_speed, :last_p_launch_angle,
                :last_p_xwoba]

for game = allgames
    # Only want to do this if it's clearly a match
    if (size(seasondata[seasondata.game_pk .== game, :], 1) == 2)
    
        towork = seasondata[seasondata.game_pk .== game, :]
        row = []
        push!(row, game)
        push!(row, towork[1, :game_date])
        push!(row, towork[1, :home_opener_odds])

        if towork[1, :HomeID] == towork[1, :player_id]
            order = [1, 2]
        else
            order = [2, 1]
        end

        homerow = towork[order[1], usefulcols]
        [push!(row, a) for a in Array(homerow)]
        # push!(row, Array(homerow))
        push!(row, towork[1, :away_opener_odds])
        awayrow = towork[order[2], usefulcols]
        [push!(row, a) for a in Array(awayrow)]
        # push!(row, Array(awayrow))

        push!(finalset, row)
    end
end

finalset.home_last5_o_hardhit_diff = finalset.home_last5_o_hardhit .- finalset.away_last5_o_hardhit;
finalset.home_last5_o_launch_speed_diff = finalset.home_last5_o_launch_speed .- finalset.away_last5_o_launch_speed;
finalset.home_last5_o_xwoba_diff = finalset.home_last5_o_xwoba .- finalset.away_last5_o_xwoba;

finalset.home_last_p_hardhit_diff = finalset.home_last_p_hardhit .- finalset.away_last_p_hardhit;
finalset.home_last_p_launch_speed_diff = finalset.home_last_p_launch_speed .- finalset.away_last_p_launch_speed;
finalset.home_last_p_xwoba_diff = finalset.home_last_p_xwoba .- finalset.away_last_p_xwoba;

In [45]:
CSV.write(joinpath(datadir, "final_format.csv"), finalset)

"../Data/2025/final_format.csv"